<a href="https://colab.research.google.com/github/somewheresy/somewhere-diffusion/blob/main/Somewhere_Diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Somewhere Diffusion
![visitors](https://visitor-badge.glitch.me/badge?page_id=somewhere_diffusion)